This is one of the Objectiv example notebooks. For more examples visit the 
[example notebooks](https://objectiv.io/docs/modeling/example-notebooks/) section of our docs. The notebooks can run with the demo data set that comes with the our [quickstart](https://objectiv.io/docs/home/try-the-demo/), but can be used to run on your own collected data as well.

All example notebooks are also available in our [quickstart](https://objectiv.io/docs/home/try-the-demo/). With the quickstart you can spin up a fully functional Objectiv demo pipeline in five minutes. This also allows you to run these notebooks and experiment with them on a demo data set.

With Objectiv you can do all your analysis and Machine Learning directly on the raw data in your SQL database.
This example shows in the simplest way possible how you can use Objectiv to create a basic feature set and use
sklearn to do machine learning on this data set. We also have an example that goes deeper into
feature engineering [here](https://objectiv.io/docs/modeling/example-notebooks/feature-engineering/).

## Getting started
If you are running this example on your own collected data, [see the instructions here](https://objectiv.io/docs/modeling/get-started-in-your-notebook/) on how to setup the database connection and get started in your favorite notebook tool.

### Import the required packages for this notebook
The open model hub package can be installed with `pip install objectiv-modelhub` (this installs Bach as well).  
If you are running this notebook from our quickstart, the model hub and Bach are already installed, so you don't have to install it separately.

In [ ]:
from modelhub import ModelHub
from sklearn import cluster

At first we have to instantiate the Objectiv DataFrame object and the model hub.

In [ ]:
# instantiate the model hub
modelhub = ModelHub(time_aggregation='%Y-%m-%d')

In [ ]:
# get the Bach DataFrame with Objectiv data
df = modelhub.get_objectiv_dataframe(start_date='2022-02-02')

We create a data set of per user all the root locations that the user clicked on. For the ins and outs on feature engineering see our feature [engineering example](https://objectiv.io/docs/modeling/example-notebooks/feature-engineering/).

In [ ]:
df['root'] = df.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id')

# root series is later unstacked and its values might contain dashes
# which are not allowed in BigQuery column names, lets replace them
df['root'] = df['root'].str.replace('-', '_')

In [ ]:
features = df[(df.event_type=='PressEvent')].groupby('user_id').root.value_counts()

In [ ]:
features.head()

In [ ]:
features_unstacked = features.unstack(fill_value=0)
# sample or not
kmeans_frame = features_unstacked
# for BigQuery the table name should be 'YOUR_PROJECT.YOUR_WRITABLE_DATASET.YOUR_TABLE_NAME'
kmeans_frame = features_unstacked.get_sample(table_name='kmeans_test', sample_percentage=50, overwrite=True)

Now we have a basic feature set that is small enough to fit in memory. This can be used with sklearn, as we
demonstrate in this example.

In [ ]:
# export to pandas now
pdf = kmeans_frame.to_pandas()

In [ ]:
pdf

In [ ]:
# do basic kmeans
est = cluster.KMeans(n_clusters=3)
est.fit(pdf)
pdf['cluster'] = est.labels_

Now you can use the created clusters on your entire data set again if you add it back to your DataFrame.
This is simple, as Bach and pandas are cooperating nicely. Your original Objectiv data now has a 'cluster'
column.

In [ ]:
kmeans_frame['cluster'] = pdf['cluster']

In [ ]:
kmeans_frame.sort_values('cluster').head()

In [ ]:
df_with_cluster = df.merge(kmeans_frame[['cluster']], on='user_id')

In [ ]:
df_with_cluster.head()

You can use this column, just as any other. For example you can now use your created clusters to group models
from the model hub by:

In [ ]:
modelhub.aggregate.session_duration(df_with_cluster, groupby='cluster').head()